### Add authentication credentials to access Big Query API

In [1]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/psparks/Documents/Springboard/GDELT Testing-31a7081b0cbc.json"

### The BigQuery Python client library provides a magic command that allows you to run queries with minimal code. To load the magic commands from the client library, paste the following code into the first cell of the notebook.

In [2]:
%load_ext google.cloud.bigquery

### The BigQuery client library provides a cell magic,  _%%bigquery_, which runs a SQL query and returns the results as a Pandas DataFrame.

In [9]:
%%bigquery france_subset
SELECT *
FROM `gdelt-bq.full.events`
WHERE ActionGeo_CountryCode = 'FR' AND MonthYear >= 201601

In [10]:
france_subset.shape

(3103885, 58)

### After downloading data, save it for future use

In [11]:
france_subset.to_csv('france_subset_1619_2.csv.bz2', header = True, compression='bz2')

### Now read it back for future data wrangling

In [12]:
import pandas as pd
import numpy as np
import sqlite3

In [13]:
france_subset = pd.read_csv('france_subset_1619_2.csv.bz2', compression='bz2')
france_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3103885 entries, 0 to 3103884
Data columns (total 59 columns):
Unnamed: 0               int64
GLOBALEVENTID            int64
SQLDATE                  int64
MonthYear                int64
Year                     int64
FractionDate             float64
Actor1Code               object
Actor1Name               object
Actor1CountryCode        object
Actor1KnownGroupCode     object
Actor1EthnicCode         object
Actor1Religion1Code      object
Actor1Religion2Code      object
Actor1Type1Code          object
Actor1Type2Code          object
Actor1Type3Code          object
Actor2Code               object
Actor2Name               object
Actor2CountryCode        object
Actor2KnownGroupCode     object
Actor2EthnicCode         object
Actor2Religion1Code      object
Actor2Religion2Code      object
Actor2Type1Code          object
Actor2Type2Code          object
Actor2Type3Code          object
IsRootEvent              int64
EventCode                int

In [14]:
france_subset.head()

,Unnamed: 0,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,...,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,0,523961397,20160324,201603,2016,2016.230100,NaN,NaN,NaN,NaN,...,FR,1,France,FR,FR,46.0000,2.00000,FR,20160324,http://www.goldcoastbulletin.com.au/news/natio...
1,1,521622559,20160316,201603,2016,2016.208200,NaN,NaN,NaN,NaN,...,FR,1,France,FR,FR,46.0000,2.00000,FR,20160316,http://www.newsmax.com/TheWire/italy-unsold-fo...
2,2,516418595,20160229,201602,2016,2016.161621,NaN,NaN,NaN,NaN,...,FR,1,France,FR,FR,46.0000,2.00000,FR,20160229,http://www.irishtimes.com/life-and-style/peopl...
3,3,522928257,20160321,201603,2016,2016.221900,NaN,NaN,NaN,NaN,...,FR,1,France,FR,FR,46.0000,2.00000,FR,20160321,http://www.ibtimes.co.uk/malaysia-palm-oil-cou...
4,4,538837301,20160510,201605,2016,2016.356200,NaN,NaN,NaN,NaN,...,-1409795,4,"French Riviera, France (general), France",FR,FR00,43.5599,7.13672,-1409795,20160510,http://movies.ndtv.com/bollywood/en-route-cann...
